In [1]:
%%javascript
Jupyter.notebook.config.update({
  'load_extensions': { 'vim_binding/vim_binding': true },
});

<IPython.core.display.Javascript object>

In [2]:
%%javascript
Jupyter.utils.load_extensions('vim_binding/vim_binding');

<IPython.core.display.Javascript object>

In [3]:
%display latex

def defvars(label, n):
    return list(var(label + '_%d' % i) for i in range(n))

def defpoly_from_basis(label, basis):
    coeffs = defvars(label,len(basis))
    poly = sum(c*x for c,x in zip(coeffs,basis))
    return (coeffs,poly)

def defpoly(label, d): 
    return defpoly_from_basis(label, list(x**i for i in range(d+1)))

def subst_poly(poly,table):
    return sum((poly.monomial_coefficient(v).subs(table) * v for v in poly.monomials()))

In [ ]:
# Simplified Gro16 randomisation (w/o x)

R.<a,b,c,alpha,beta,gamma,delta,f> = LaurentPolynomialRing(SR)

sigma_1 = [a,c,alpha,beta,delta,f]
sigma_2 = [b,beta,gamma,delta,f]

A_coeffs,A = defpoly_from_basis('A',sigma_1)
B_coeffs,B = defpoly_from_basis('B',sigma_2)
C_coeffs,C = defpoly_from_basis('C',sigma_1)

V1_1_l = (a * b - alpha*beta - f - c * delta)
V1_1_r = (A * B - alpha*beta - f - C * delta)

print("Can we solve V1 alone:")
print(len(solve(V1_1_r.coefficients(), A_coeffs + B_coeffs + C_coeffs, solution_dict=True)) > 0)

sols = solve((V1_1_r - V1_1_l).coefficients(), A_coeffs + B_coeffs + C_coeffs, solution_dict=True)
show(sols)
print("Solutions:")
for s in sols:    
    show(subst_poly(A,s)); show(subst_poly(B,s)); show(subst_poly(C,s))

In [ ]:
# (???) Simplified Gro16 randomisation + extra coefficients in V1

R.<alpha,beta,gamma,delta,f> = LaurentPolynomialRing(SR)

var('a,b,c')
sigma_1 = [a,c,alpha,beta,delta,f]
sigma_2 = [b,beta,gamma,delta,f]

A_coeffs,A = defpoly_from_basis('A',sigma_1)
B_coeffs,B = defpoly_from_basis('B',sigma_2)
C_coeffs,C = defpoly_from_basis('C',sigma_1)

V1_1_l = ((a + alpha*delta) * (b + beta*delta) - alpha*beta*delta**2 - f*gamma*delta - c * delta)
V1_1_r = V1_1_l(a = A, b = B, c = C)

sols = solve((V1_1_r - V1_1_l).coefficients(), A_coeffs + B_coeffs + C_coeffs, solution_dict=True)
show(sols)

print("Solutions:")
for s in sols:    
    show(subst_poly(A,s)); show(subst_poly(B,s)); show(subst_poly(C,s))

In [ ]:
# Full Gro16 CRS, w/o representation of a,b,c

n = 2
m = 1
l = 0

var('a,b,c')
var('alpha,beta,gamma,delta,x') # CRS trapdoors

as1_phi = list(var('as_1_%d' % i) for i in range(l+1))
as2_phi = list(var('as_2_%d' % i) for i in range(l+1))
R.<a,b,c,alpha,beta,gamma,delta,x> = LaurentPolynomialRing(SR)
t_coeffs,t = defpoly('t',n)

us = []
uscoeffs = []
vs = []
vscoeffs = []
ws = []
wscoeffs = []

for i in range(m+1):
    u_coeffs,u = defpoly('u_%d'%i,n-1)
    us.append(u); uscoeffs.append(u_coeffs)
    
    v_coeffs,v = defpoly('v_%d'%i,n-1)
    vs.append(v); vscoeffs.append(v_coeffs)
    
    w_coeffs,w = defpoly('w_%d'%i,n-1)
    ws.append(w); wscoeffs.append(w_coeffs)

# We don't capture the fact that u_i are linearly independent though
    
sigma_1 = [a,c,alpha,beta,delta] + \
            list(x**i for i in range(n)) + \
            list((beta*us[i] + alpha*vs[i] + ws[i])/gamma for i in range(l+1)) + \
            list((beta*us[i] + alpha*vs[i] + ws[i])/delta for i in range(l+1, m+1)) + \
            list(x**i * t / delta for i in range(n-1))
sigma_2 = [b,beta,gamma,delta] + list(x**i for i in range(n))

f1 = sum(as1_phi[i] * (beta * us[i] + alpha * vs[i] + ws[i]) for i in range(len(as1_phi)))
f2 = sum(as2_phi[i] * (beta * us[i] + alpha * vs[i] + ws[i]) for i in range(len(as2_phi)))

A_coeffs,A = defpoly_from_basis('A',sigma_1)
B_coeffs,B = defpoly_from_basis('B',sigma_2)
C_coeffs,C = defpoly_from_basis('C',sigma_1)

V1_1_l = a * b - alpha*beta - f1 - c * delta
V1_1_r = A * B - alpha*beta - f1 - C * delta

print("Can we solve V1 alone from the CRS:")
sol1 = solve(V1_1_r.coefficients(), A_coeffs + B_coeffs + C_coeffs + as1_phi, solution_dict=True)
print(len(sol1) > 0)
if (len(sol1) > 0):
    print(sol1)

print("Rand equaion solutions:")
sols = solve((V1_1_r - V1_1_l).coefficients(), A_coeffs + B_coeffs + C_coeffs, solution_dict=True)
print(len(sols))
show(sols[0])
for s in sols:    
    show(subst_poly(A,s))
    show(subst_poly(B,s))
    show(subst_poly(C,s))

In [ ]:
show(sols[0])
for s in sols:    
    show(subst_poly(A,s))
    show(subst_poly(B,s))
    show(subst_poly(C,s))

In [ ]:
# Full Gro16 CRS, WITH representation of a,b,c

n = 2
m = 1
l = 0

var('a_r, b_r, c_r')
var('alpha,beta,gamma,delta,x') # CRS trapdoors
var('r,s') # random variables we don't know

as_phi = list(var('a_%d' % i) for i in range(l+1))
as_wit = list(var('a_%d' % i) for i in range(l+1, m+1))
as_all = as_phi + as_wit
rvars = [alpha,beta,gamma,delta,x]+as_wit+[a_r,b_r,c_r]
# How to pass as_wit LIST to the angle brackets? Who knows...
R.<alpha,beta,gamma,delta,x,r,s,a_r,b_r,c_r,a_1> = LaurentPolynomialRing(SR)
t_coeffs,t = defpoly('t',n)
h_coeffs,h = defpoly('h',n-2)

us = []
uscoeffs = []
vs = []
vscoeffs = []
ws = []
wscoeffs = []

for i in range(m+1):
    u_coeffs,u = defpoly('u_%d'%i,n-1)
    us.append(u); uscoeffs.append(u_coeffs)
    
    v_coeffs,v = defpoly('v_%d'%i,n-1)
    vs.append(v); vscoeffs.append(v_coeffs)
    
    w_coeffs,w = defpoly('w_%d'%i,n-1)
    ws.append(w); wscoeffs.append(w_coeffs)

allcoeffs = flatten(uscoeffs)+flatten(vscoeffs)+flatten(wscoeffs)+t_coeffs    
    

qap_eq = sum(as_all[i] * us[i] for i in range(m+1)) * sum(as_all[i] * vs[i] for i in range(m+1)) - \
            sum(as_all[i] * ws[i] for i in range(m+1)) - h * t

sigma_1 = [alpha,beta,delta] + \
            list(x**i for i in range(n)) + \
            list((beta*us[i] + alpha*vs[i] + ws[i])/gamma for i in range(l+1)) + \
            list((beta*us[i] + alpha*vs[i] + ws[i])/delta for i in range(l+1, m+1)) + \
            list(x**i * t / delta for i in range(n-1))
sigma_2 = [beta,gamma,delta] + list(x**i for i in range(n))

a = alpha + sum(as_all[i] * us[i] for i in range(m+1)) + r * delta
b = beta + sum(as_all[i] * vs[i] for i in range(m+1)) + s * delta
c = sum(as_all[i] * (beta * us[i] + alpha * vs[i] + ws[i]) + h * t for i in range(l+1, m+1)) / delta + \
    a * s + b * r - r * s * delta
f = sum(as_all[i] * (beta * us[i] + alpha * vs[i] + ws[i]) for i in range(0,l+1))

#sigma_1.insert(0,c)
#sigma_1.insert(0,a)
#sigma_2.insert(0,b)

A_coeffs,A = defpoly_from_basis('A',sigma_1)
B_coeffs,B = defpoly_from_basis('B',sigma_2)
C_coeffs,C = defpoly_from_basis('C',sigma_1)

#inv_vars = list(var('Ainv_0,Ainv_1,Binv_0,Cinv_0,Cinv_1'))
#inv_vars = list(var('Ainv_0,Binv_0,Cinv_1'))
#inv_eq = A_0 * Ainv_0 + A_1 * Ainv_1 + B_0 * Binv_0 + C_0 * Cinv_0 + C_1 * Cinv_1 - 1
#inv_eqs = [A_0 * Ainv_0 - 1, B_0 * Binv_0 - 1, C_1 * Cinv_1 - 1]

#V1_1_l = a * b - alpha*beta - f - c * delta
V1_1_r = A * B - alpha*beta - f - C * delta

#R1 = R.quotient(alpha * beta - gamma); print(R1)
#show(tmpvar1)
#show(tmpvar2)
#show(R1(tmpvar2).lift())

# TODO QAP_EQ is of different kind... that's why we don't get the solution. How to properly encode the system?
# Represent QAP as equations over A,B,C coeffs using CRS bits?

#system = V1_1_r.coefficients() + inv_eqs + [qap_eq]
#show(system)
#sols = solve(system, A_coeffs + B_coeffs + C_coeffs + inv_vars, solution_dict=True)
#sols = solve(V1_1_r.coefficients(), A_coeffs + B_coeffs + C_coeffs, solution_dict=True)
#print(len(sols))


#show(a)
#R1 = R.quotient(a - a_r); print(R1)
#print("Solutions V1 + V2:")
#for s in sols:    
#    show(subst_poly(Rinv(inv_eq),s))
##    show(subst_poly(A,s))
#    show(subst_poly(B,s))
#    #show(R1(subst_poly(B,s)).lift())
##    show(subst_poly(C,s))
##    show(subst_poly(A2,s)); show(subst_poly(B2,s)); show(subst_poly(C2,s)); show(subst_poly(B12,s))

In [ ]:
show(sigma_1)
show(sigma_2)
show(V1_1_r)
show(V1_1_r.monomial_coefficient(alpha * beta))
show(V1_1_r.monomial_coefficient(alpha * alpha))
show(V1_1_r.monomial_coefficient(beta * beta))
show(V1_1_r.monomial_coefficient(1 / (delta)))
show(V1_1_r.monomial_coefficient(alpha / (delta)))
show(V1_1_r.monomial_coefficient(alpha / (delta ** 2)))

In [ ]:
print(sols)

In [ ]:
# Lip19 randomisability

R.<a,b,c,b1,z,alpha,beta,gamma,delta,eta,f,tau> = PolynomialRing(SR)

sigma_1 = [a,c,b1,alpha,gamma,delta,alpha*z,beta*z,tau]
sigma_2 = [b,alpha,delta,eta,tau]

A2_coeffs,A2 = defpoly_from_basis('A',sigma_1)
B2_coeffs,B2 = defpoly_from_basis('B',sigma_2)
B12_coeffs,B12 = defpoly_from_basis('B1',sigma_1)
C2_coeffs,C2 = defpoly_from_basis('C',sigma_1)


V1_1_l = ((a + gamma) * (b + delta) - gamma*delta - f - c * alpha)
V1_1_r = V1_1_l(a = A2, b = B2, c=C2)

print("V1: ")
show(V1_1_l)

sols = solve((V1_1_r - V1_1_l).coefficients(), A2_coeffs + B2_coeffs + C2_coeffs, solution_dict=True)
print("Solutions V1:")
for s in sols:    
    show(subst_poly(A2,s)); show(subst_poly(B2,s)); show(subst_poly(C2,s))

V1_2_l = (b1 - (z + tau) * b)
V1_2_r = V1_2_l(b = B2, b1 = B12)

print ("V2: ")
show(V1_2_l)

sols = solve((V1_2_r - V1_2_l).coefficients(), A2_coeffs + B2_coeffs + C2_coeffs + B12_coeffs, solution_dict=True)
print("Solutions V2:")
for s in sols:    
    show(subst_poly(B2,s)); show(subst_poly(B12,s))

sols = solve((V1_2_r - V1_2_l).coefficients() + (V1_1_l - V1_1_r).coefficients(), A2_coeffs + B2_coeffs + B12_coeffs + C2_coeffs, solution_dict=True)

print("Solutions V1 + V2:")
for s in sols:    
    show(subst_poly(A2,s)); show(subst_poly(B2,s)); show(subst_poly(C2,s)); show(subst_poly(B12,s))


In [ ]:
# Find random-detecting polynomial

var('a_r, b_r, c_r')

sigma_1.insert(0,A3)
sigma_1.insert(0,C3)
sigma_2.insert(0,B3)

# maximum degree of pairing allowed
pD = 2

rgs = []
rgs_coeffs = []
rhs = []
rhs_coeffs = []

abc_coeffs = []

veq = 0

for i in range(pD):
    rgi_coeffs,rgi = defpoly_from_basis('rg_%d' % i, sigma_1)
    rgs.append(rgi); rgs_coeffs.extend(rgi_coeffs)
    
    rhi_coeffs,rhi = defpoly_from_basis('rh_%d' % i, sigma_2)
    rhs.append(rhi); rhs_coeffs.extend(rhi_coeffs)
    
    abc_coeffs.extend(rgi_coeffs[:4])
    abc_coeffs.extend(rhi_coeffs[:2])
    
    veq = veq + rgi * rhi

print(abc_coeffs)
#sols = solve(veq.coefficients(), rgs_coeffs + rhs_coeffs, solution_dict=True)

In [ ]:
show(sols[0])
rgs_substituted = map(lambda x: subst_poly(x,sols[0]), rgs)
rhs_substituted = map(lambda x: subst_poly(x,sols[0]), rhs)

show(rgs_substituted)
show(rhs_substituted)

In [63]:
# GM17 (old) CRS to simplify the attempts to prove it sound without V2

n = 2
m = 1
l = 0

var('alpha,beta,gamma,x') # CRS trapdoors
var('r') # proof random

s_phi = list(var('s_%d' % i) for i in range(l+1))
s_wit = list(var('s_%d' % i) for i in range(l+1, m+1))
s_all = s_phi + s_wit
R.<alpha,beta,gamma,x,r,s_1> = LaurentPolynomialRing(SR)
t_coeffs,t = defpoly('t',n)
h_coeffs,h = defpoly('h',n-2)

us = []
uscoeffs = []
vs = []
vscoeffs = []
ws = []
wscoeffs = []

for i in range(m+1):
    u_coeffs,u = defpoly('u_%d'%i,n-1)
    us.append(u); uscoeffs.append(u_coeffs)
    
    v_coeffs,v = defpoly('v_%d'%i,n-1)
    vs.append(v); vscoeffs.append(v_coeffs)
    
    w_coeffs,w = defpoly('w_%d'%i,n-1)
    ws.append(w); wscoeffs.append(w_coeffs)

sigma_1 = [alpha,beta,gamma*t,gamma**2*t**2,(alpha+beta)*gamma*t] + \
            list(gamma * x**i for i in range(n)) + \
            list(gamma ** 2 * t * x**i for i in range(n)) + \
            list(gamma * ws[i] + (alpha+beta) * us[i] for i in range(l+1)) + \
            list(gamma ** 2 * ws[i] + gamma * (alpha+beta) * us[i] for i in range(l+1,m+1))
sigma_2 = [beta,gamma,gamma*t] + list(gamma * x**i for i in range(n))

var('A_1,A_2,A_3,A_4,A_5')
A6_vars = defvars('A_6', n)
A7_vars = defvars('A_7', n)
A8_vars = defvars('A_8', l+1)
A9_vars = defvars('A_9', l+1)

var('B_1,B_2,B_3')
B4_vars = defvars('C_4', n)

var('C_1,C_2,C_3,C_4,C_5')
C6_vars = defvars('C_6', n)
C7_vars = defvars('C_7', n)
C8_vars = defvars('C_8', l+1)
C9_vars = defvars('C_9', m - l)

A_vars = [A_1,A_2,A_3,A_4,A_5] + A6_vars + A7_vars + A8_vars + A9_vars
B_vars = [B_1,B_2,B_3] + B4_vars
C_vars = [C_1,C_2,C_3,C_4,C_5] + C6_vars + C7_vars + C8_vars + C9_vars

ABC_vars = A_vars + B_vars + C_vars

f = sum(s_all[i] * (beta * us[i] + alpha * vs[i] + ws[i]) for i in range(0,l+1))

A = sum(c*x for c,x in zip(A_vars,sigma_1))
B = sum(c*x for c,x in zip(B_vars,sigma_2))
C = sum(c*x for c,x in zip(C_vars,sigma_1))

V1_1_r = (A + alpha) * (B + beta) - alpha*beta - f - C

In [79]:
#print(V1_1_r.parent())
#show(V1_1_r)
mons = [alpha * beta, alpha, beta, alpha ** 2, beta ** 2, \
        gamma**3 * x**(3*n), \
        gamma**3 * x**(3*n-1), \
        gamma**3 * x**(3*n-2), \
        gamma**3 * x**(2*n), \
        gamma**3 * x**(2*n-1), \
        gamma**3 * x**(2*n-2), \
        beta * gamma**2 * x**(2*n), \
        beta * gamma**2 * x**(2*n-1), \
        alpha * gamma**2 * x**(2*n), \
        alpha * gamma**2 * x**(2*n-1), \
        beta**2 * gamma * x**n, \
        beta*alpha * gamma * x**n, \
        alpha**2 * gamma * x**n, \
       ]
system = list(V1_1_r.monomial_coefficient(mon) for mon in mons)
system_vars = list(set(flatten(list(x.variables() for x in system))))
relevant_vars = list(filter(lambda y: y in system_vars, ABC_vars))
solve(system,relevant_vars)

[[A_1 == ((r544 - 1)*r545 + r544)/(r545 + 1), A_2 == r544, A_4 == 0, A_5 == 0, A_7_0 == 0, A_7_1 == 0, A_8_0 == -r544/u_0_0, A_9_0 == 0, B_1 == r545, B_2 == r546, B_3 == r547, C_4_0 == r548, C_4_1 == r549, C_1 == r550, C_2 == -s_0*u_0_0 + s_0*v_0_0 + r550, C_8_0 == -(s_0*v_0_0 + r550)/u_0_0], [A_1 == ((r551 - 1)*r554 + r551)/(r554 + 1), A_2 == r551, A_4 == 0, A_5 == 0, A_7_0 == r552, A_7_1 == 0, A_8_0 == -r551/u_0_0, A_9_0 == r553, B_1 == r554, B_2 == r555, B_3 == 0, C_4_0 == -r555, C_4_1 == 0, C_1 == r556, C_2 == -s_0*u_0_0 + s_0*v_0_0 + r556, C_8_0 == -(s_0*v_0_0 + r556)/u_0_0], [A_1 == ((r557 - 1)*r559 + r557)/(r559 + 1), A_2 == r557, A_4 == 0, A_5 == 0, A_7_0 == 0, A_7_1 == 0, A_8_0 == -r557/u_0_0, A_9_0 == r558, B_1 == r559, B_2 == r560, B_3 == 0, C_4_0 == r561, C_4_1 == 0, C_1 == r562, C_2 == -s_0*u_0_0 + s_0*v_0_0 + r562, C_8_0 == -(s_0*v_0_0 + r562)/u_0_0]]